In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

df = pd.read_csv('preprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(3333, random_state=42)) #


In [3]:
x = df['Lyric']
y = df['Genre']

In [4]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(df.Genre.values)
np.unique(labels)

array([0, 1, 2])

In [5]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(df.Lyric.values, 
                                                labels, 
                                                random_state=42, 
                                                test_size=0.1, 
                                                shuffle=True)

In [6]:
token = Tokenizer()


In [7]:
token.fit_on_texts(xtrain)
seq = token.texts_to_sequences(xtrain)


In [8]:
pad_seq = pad_sequences(seq,maxlen=300)


In [9]:
seq_test = token.texts_to_sequences(xtest)
pad_seq_test = pad_sequences(seq_test,maxlen=300)


In [10]:
vocab_size = len(token.word_index)+1


In [11]:
embedding_vector = {}
f = open('glove.6B.100d.txt',encoding='utf8')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400001it [00:10, 39684.35it/s]


In [12]:
embedding_matrix = np.zeros((vocab_size,100))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 27966/27966 [00:00<00:00, 532835.64it/s]


In [13]:
model = Sequential()

In [14]:
model.add(Embedding(vocab_size,100,weights = [embedding_matrix],input_length=300,trainable = False))


In [15]:
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.add(Dense(32,activation = 'relu'))


In [17]:
model.add(Dense(3,activation = 'sigmoid'))


In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
pad_seq.shape

(8999, 300)

In [20]:
from keras.utils import np_utils

ytrain_encode = np_utils.to_categorical(ytrain)
ytest_encode = np_utils.to_categorical(ytest)

In [21]:
history = model.fit(pad_seq,ytrain_encode,epochs = 5,batch_size=16)




Epoch 1/5
8999/8999 [==============================] - 525s 58ms/step - loss: 0.5459 - acc: 0.7103
Epoch 2/5
8999/8999 [==============================] - 525s 58ms/step - loss: 0.4852 - acc: 0.7484
Epoch 3/5
8999/8999 [==============================] - 524s 58ms/step - loss: 0.4665 - acc: 0.7630
Epoch 4/5
8999/8999 [==============================] - 521s 58ms/step - loss: 0.4582 - acc: 0.7649
Epoch 5/5
8999/8999 [==============================] - 521s 58ms/step - loss: 0.4451 - acc: 0.7767


In [22]:
from sklearn.metrics import classification_report

y_pred = model.predict(pad_seq_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(ytest, y_pred_bool))

1000/1000 [==============================] - 6s 6ms/step
              precision    recall  f1-score   support

           0       0.80      0.77      0.79       334
           1       0.50      0.49      0.49       323
           2       0.64      0.68      0.66       343

    accuracy                           0.65      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.65      0.65      0.65      1000



In [23]:
values = history.history


In [24]:
values

{'loss': [0.5458940031436009,
  0.48521052355739275,
  0.4665195840883949,
  0.45820862021376285,
  0.4451483717619386],
 'acc': [0.7103382036518767,
  0.7483794696861696,
  0.7630477651700349,
  0.7648627810815213,
  0.7766789110028249]}

In [25]:
val_loss = values['val_loss']
training_loss = values['loss']
training_acc = values['acc']
validation_acc = values['val_acc']
epochs = range(5)

plt.plot(epochs,val_loss,label = 'Validation Loss')
plt.plot(epochs,training_loss,label = 'Training Loss')
plt.title('Epochs vs Loss')
plt.legend()
plt.show()

KeyError: 'val_loss'

In [ ]:
plt.plot(epochs,validation_acc,label = 'Validation Accuracy')
plt.plot(epochs,training_acc,label = 'Training Accuracy')
plt.title('Epochs vs Accuracy')
plt.legend()
plt.show()